In [ ]:
# pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ProjecTweets").getOrCreate()

23/10/21 09:51:53 WARN Utils: Your hostname, Kagan-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.5 instead (on interface en0)
23/10/21 09:51:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/21 09:51:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load the data
df = spark.read.csv("/Users/kagantimur/Desktop/CA2/ProjectTweets.csv", inferSchema=True, header=True)

In [3]:
# See the data
df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- 1467810369: long (nullable = true)
 |-- Mon Apr 06 22:19:45 PDT 2009: string (nullable = true)
 |-- NO_QUERY: string (nullable = true)
 |-- _TheSpecialOne_: string (nullable = true)
 |-- @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D: string (nullable = true)



In [4]:
df = df.drop("0")

In [5]:
from pyspark.sql.functions import col
df = df.withColumnRenamed("1467810369", "ID")\
       .withColumnRenamed("Mon Apr 06 22:19:45 PDT 2009", "Date")\
       .withColumnRenamed("NO_QUERY", "Flag")\
       .withColumnRenamed("_TheSpecialOne_", "User")\
       .withColumnRenamed("@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D", "Text")

In [7]:
df = df.withColumnRenamed("@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", "Text")

In [8]:
df.show()

+----------+--------------------+--------+---------------+--------------------+
|        ID|                Date|    Flag|           User|                Text|
+----------+--------------------+--------+---------------+--------------------+
|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nope t...|
|1467812025|Mon Apr 06 22:20:...|NO_QUERY|        mimismo|@twittera que me ...|
|1467812416|Mon Apr 06 22:20:...|NO_QUER

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import TimestampType
from dateutil import parser

In [11]:
# Define a UDF to parse date-time strings and apply time zone conversion
def parse_date_with_tz(date_str):
    if date_str is not None:
        return parser.parse(date_str, tzinfos={"PDT": -7})  # Adjust time zone offset as needed
    return None

parse_date_udf = udf(parse_date_with_tz, TimestampType())

In [12]:
# Apply the UDF to the 'Date' column
df = df.withColumn("Date", parse_date_udf(col("Date")))

# Show the DataFrame with parsed date-time
df.show(truncate=False)

+----------+-------------------+--------+---------------+---------------------------------------------------------------------------------------------------------------------+
|ID        |Date               |Flag    |User           |Text                                                                                                                 |
+----------+-------------------+--------+---------------+---------------------------------------------------------------------------------------------------------------------+
|1467810672|2009-04-06 23:19:56|NO_QUERY|scotthamilton  |is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!      |
|1467810917|2009-04-06 23:20:00|NO_QUERY|mattycus       |@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                            |
|1467811184|2009-04-06 23:20:04|NO_QUERY|ElleCTF        |my whole body feels itchy and like its on fire                 

In [14]:
df.coalesce(1).write.option("header", "true").option("delimiter", ",").csv("/Users/kagantimur/Desktop/CA2/Organized_ProjectTweets.csv")

23/10/21 10:03:49 ERROR FileFormatWriter: Aborting job bee6014c-7d00-4ac3-ad35-fe480675d9d9.
java.io.FileNotFoundException: File file:/Users/kagantimur/Desktop/CA2/Organized_ProjectTweets.csv/_temporary/0/.DS_Store does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:390)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:341)
	at org.apache.hadoop.fs.RawLocalFileSystem.rename(RawLocalFileSystem.java:505)
	at org.apache.hadoop.fs.ChecksumFileSystem.rename(ChecksumFileSystem.java:700)
	at org.apache.hadoop.hive.ql.io.ProxyLocalFileSystem.rename(ProxyLocalFileSystem.java:34)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.mergePaths(FileOutputCommitter.java:476)
	at org.apache.

Py4JJavaError: An error occurred while calling o69.csv.
: java.io.FileNotFoundException: File file:/Users/kagantimur/Desktop/CA2/Organized_ProjectTweets.csv/_temporary/0/.DS_Store does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:390)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:341)
	at org.apache.hadoop.fs.RawLocalFileSystem.rename(RawLocalFileSystem.java:505)
	at org.apache.hadoop.fs.ChecksumFileSystem.rename(ChecksumFileSystem.java:700)
	at org.apache.hadoop.hive.ql.io.ProxyLocalFileSystem.rename(ProxyLocalFileSystem.java:34)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.mergePaths(FileOutputCommitter.java:476)
	at org.apache.hadoop.mapreduce.lib.output.FileOut

In [ ]:
# pip install pyspark mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import mysql.connector

In [8]:
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "kda2023!",
    "database": "ProjectTweets",
    "port": "3306"
}

In [10]:
# Replace this with the path to your CSV file
csv_file = "/Users/kagantimur/Desktop/CA2/Organized_ProjectTweets.csv"

In [11]:
# Read the CSV file into a Spark DataFrame
df = spark.read.csv(csv_file, header=True, inferSchema=True)

In [12]:
# Connect to the MySQL database
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

In [13]:
# SQL query to create a table
create_table_query = """
CREATE TABLE Tweets (
    ID INT(11) NOT NULL AUTO_INCREMENT,
    Date TIMESTAMP NULL DEFAULT NULL,
    Flag VARCHAR(250),
    User VARCHAR(128) DEFAULT NULL,
    Text TEXT,
    PRIMARY KEY (ID)
) ENGINE=InnoDB AUTO_INCREMENT=1 DEFAULT CHARSET=utf8;
"""

# Execute the SQL query
cursor.execute(create_table_query)
conn.commit()


In [14]:
# Close the previous result set (if any)
if cursor.with_rows:
    cursor.fetchall()

# SQL query to list all tables in the database
show_tables_query = "SHOW TABLES;"

# Execute the SQL query
cursor.execute(show_tables_query)

# Fetch all the table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

Tweets


In [15]:
# Execute the SQL query to get column names
cursor.execute("SHOW COLUMNS FROM tweets")

# Fetch the results
columns = cursor.fetchall()

# Display the column names
for column in columns:
    print(column[0])

ID
Date
Flag
User
Text


In [16]:
import mysql.connector

def store_data(Date, Flag, User, Text):
    cursor = conn.cursor()
    insert_query = "INSERT INTO Tweets (Date, Flag, User, Text) VALUES (%s, %s, %s, %s)"
    cursor.execute(insert_query, (Date, Flag, User, Text))
    conn.commit()
    return

In [ ]:
#cursor.close()
#conn.close()

In [17]:
# Check the number of rows in the "Tweets" table
cursor.execute("SELECT COUNT(*) FROM Tweets")
row_count = cursor.fetchone()[0]

if row_count > 0:
    print("The 'Tweets' table contains data.")
else:
    print("The 'Tweets' table is empty.")

The 'Tweets' table is empty.


In [18]:
# Execute the SQL query to count the rows
cursor.execute("SELECT COUNT(*) FROM tweets")

# Fetch the result
row_count = cursor.fetchone()[0]

# Display the row count
print("Number of rows in 'tweets' table:", row_count)

Number of rows in 'tweets' table: 0
